In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [11]:
info_dir = '/search/odin/publicData/CloudS/rank/infos/video/common'
info_dir = '/home/gezi/tmp/rank/exps/video/dlrm'
hour = 2020031922
info_path = f'{info_dir}/infos/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df.head()

/home/gezi/tmp/rank/exps/video/dlrm/infos/2020031922/valid.csv


,mid,duration,ori_lr_score,abtest,rea,page_time,product_data,distribution,impression_time,position,video_time,read_completion_rate,docid,pred,pred_click,pred_dur,dur
0,9c14860459045620835,0,0.166456,4,990,320,sgsapp,3057,1584625607,4,351,0.0,20306e042vqap8,0.035089,0.163818,0.240075,0
1,b443863668046244232,0,0.193433,6,990,21,sgsapp,3116,1584627261,8,765,0.0,2031910K3AhraJ,0.009089,0.055680,0.259196,0
2,aec7869011033167955,0,0.309933,5,954,149,sgsapp,1445,1584626677,9,414,0.0,20313l040FgQJK,0.013989,0.059083,0.428438,0
3,9ec31584625201300|358ff6,330,1.333333,4,913,715,sgsapp,8160,1584625319,5,462,1.0,20220g0K1SOtnp,0.029489,0.189245,0.143089,330
4,3751863272036611151,-1,0.180933,5,913,12,sgsapp,1439,1584625379,6,291,0.0,20319a0K0Rcs8j,0.079389,0.313049,0.231686,-1


In [12]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 269/269 [00:09<00:00, 27.35it/s]


In [13]:
pd.set_option('display.max_rows', 1000)

In [14]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [15]:
mean_duration = df[df.dur > 0].dur.mean()

In [16]:
mean_duration

160.23390518354546

In [17]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [18]:
cold_mids = set(df[df.cold==True].mid.values)

In [19]:
len(set(df.mid.values))

10568

In [20]:
len(cold_mids)

563

In [21]:
def debug(mid=None, cold=False):
  if mid is None:
    if not cold:
      mid = df.mid.sample(1).values[0]
    else:
      mid = df[df.mid.isin(cold_mids)].mid.sample(1).values[0]
  print('mid:', mid)
  cols = ['position', 'docid', 'title', 'dur', 'ori_lr_score', 'pred', 'pred_click', 'pred_dur', 'cold']
  if 'hour' in df.columns:
    cols += ['hour']
  d = df[df.mid==mid]
  start_time = d.impression_time.min()
  d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
  cols += ['time']
  d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#   return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
  return d

In [30]:
d = debug()
d

mid: 6288865580031319606


,position,docid,title,dur,ori_lr_score,pred,pred_click,pred_dur,cold,time
0,0,20304k0K2E3Mot,七米深的龙王坑，抽干后发现一斤四百多的大石斑,0,0.394723,0.096989,0.255527,0.449132,False,0
1,2,20315i0K4agJcP,小明组团抽五米深大水坑，抽干后村花不淡定了，鱼群全部被困坑底,0,0.370940,0.091189,0.259942,0.398674,False,0
2,4,2030160K04OKto,灯塔下4米深的大水坑困住成群大野货，小伙看呆了,0,0.360971,0.071289,0.199047,0.459984,False,0
3,6,20223b042o3uYp,孤岛赶海遇猛货，伸手一抓被“咬”出血，却被老渔民制服,0,0.328329,0.077889,0.246866,0.350185,False,0
4,8,20315i0K0rKkQE,阿文赶海撞上大运了，一来就抓到只中华锦绣龙虾，这样抓货太过瘾,94,0.325035,0.053089,0.151503,0.500961,False,0
5,10,open_doc_prod19036856,大神利用古老的方法打造最秘密的地下竹屋，看一遍爽一天,80,0.299560,0.058489,0.167869,0.475814,False,0
6,0,20316h0K2r4sWV,阿文发现一处赶海宝地，却看见成片死鱼搁浅水塘，赶紧拿回去抢救,0,0.337223,0.074189,0.207708,0.450021,False,1
7,2,20319k0K4wcdts,丁俊晖这一杆长台就像跟踪导弹一样精准，连自己都不敢相信,212,0.322332,0.108789,0.293667,0.408865,False,1
8,4,20315g0K2fF14G,近海大坑好货真多，刚抽干就发现庞然大物，牢牢的吸住小川不放,0,0.319568,0.038289,0.120401,0.481452,False,1
9,6,2031840K095zaW,这就是韩国顶级大厨的刀功 不到十分钟就做好三斤黑鲷刺身,0,0.318659,0.089889,0.213500,0.563235,False,1


In [23]:
d = d.sort_values('ori_lr_score', ascending=False)
d['idx'] = range(len(d))
d[d.dur!=0 ].idx.mean()

10.0

In [ ]:
d = d.sort_values('pred', ascending=False)
d['idx'] = range(len(d))
d[d.dur!=0 ].idx.mean()

In [ ]:
roc_auc_score(d.dur !=0, d.ori_lr_score)

In [ ]:
roc_auc_score(d.dur !=0, d.pred)

In [ ]:
d = d.sort_values('ori_lr_score', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.ori_lr_score.values[:20])

In [ ]:
d = d.sort_values('pred', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.pred.values[:20])

In [ ]:
d = d.sort_values('pred_click', ascending=False)
roc_auc_score((d.dur != 0).values[:20], d.pred_click.values[:20])

In [ ]:
qgrid.set_grid_option('forceFitColumns', False)

In [ ]:
qgrid.show_grid(d)

In [ ]:
len(set(df.mid.values))

In [ ]:
d = df[['mid', 'impression_time']].groupby('mid', as_index=False).agg({'impression_time':'nunique'})

In [ ]:
d = df[['mid', 'impression_time']].groupby('mid', as_index=False).agg({'impression_time':'count'})

In [ ]:
df['mid'].value_counts()

In [ ]:
len(d[d.impression_time<6])

In [ ]:
px.histogram(d[d.impression_time<100], 'impression_time')

In [ ]:
df[['mid', 'impression_time']]

In [ ]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [ ]:
impressions = set(df[['mid', 'impression_time', 'impression']].groupby('mid', as_index=False).first().impression.values)

In [ ]:
len(df[df.impression.isin(impressions)].sort_values('mid'))

In [ ]:
len(set(df[df.cold==True].docid.values))

In [ ]:
len(df[df.cold==True])

In [ ]:
util.cold_starts

In [ ]:
df[df.cold==True].head()

In [ ]:
d = df[['mid', 'dur']]
d.dur = d.dur != 0

In [ ]:
d2 = d.groupby(['mid']).max()

In [ ]:
d2[d2.duration==False].reset_index().mid